# Projet sur l'autocomplétion — première partie

Le projet de 2017-2018 porte sur l'autocomplétion d'un champ de formulaire. L'objectif *In fine* est de proposer une complétion de requête muli-mots à la Google ou à la façon d'un clavier de smartphone : si je tape « machine à » le système devra me proposer « café, laver ou coudre » par exemple. L'idée est d'explorer et d'implémenter une technique pour y arriver et/ou de proposer de la complétion dans une langue autre que le français ou l'anglais. 

Dans un premier temps nous allons nous limiter à une complétion de mot à partir des deux premiers caractères saisis par un utilisateur.
Nous allons surtout mettre en place les technos permettant la complétion sur une interface web.   
Pour cela nous utiliserons une bibliothèque JavaScript côté client : [Jquery UI](http://jqueryui.com/autocomplete) et un serveur Python en mode REST pour les données : [Flask](http://flask.pocoo.org/) 

##  Les outils
### La lib [``autocomplete``](http://jqueryui.com/autocomplete/)

Cette bibliothèque va nous épargner tout ou presque du développement JS nécessaire à notre application.

Malheureusement le JS ne pourra pas être testé dans ce notebook, vous devrez copier-coller le code ```html``` et ``js`` dans un fichier et l'ouvrir avec un navigateur pour tester.   
Commençons par ce code extrait de [http://api.jqueryui.com/autocomplete/#entry-examples](http://api.jqueryui.com/autocomplete/#entry-examples)

In [ ]:
<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <title>autocomplete demo</title>
  <link rel="stylesheet" href="http://code.jquery.com/ui/1.12.1/themes/smoothness/jquery-ui.css">
  <script src="http://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="http://codte.jquery.com/ui/1.12.1/jquery-ui.js"></script>
</head>
<body>
 
<label for="autocomplete">Select a programming language: </label>
<input id="autocomplete">
 
<script>
$( "#autocomplete" ).autocomplete({
  source: [ "c++", "java", "php", "coldfusion", "javascript", "asp", "ruby" ]
});
</script>
 
</body>
</html>


Ça fonctionne évidemmen mais les données de complétion sont écrites dans le fichier. Pour une liste de quelques items très bien mais pour un lexique entier ce n'est pas envisageable.  
La bibliothèque prévoit de pouvoir faire appel à une ressource externe qui renvoie un fichier ``JSON``. Il s'agit d'un script ``php`` dans l'exemple [http://jqueryui.com/autocomplete/#remote](http://jqueryui.com/autocomplete/#remote) mais ici nous travaillerons en  Python.

### ``Flask``
    

``Flask`` est un micro framework de développement web. Ici nous l'emploierons uniquement pour envoyer (ou servir) des données à notre page html.   
Nous allons nous contenter d'un code minimal pour commencer, voir la [doc](http://flask.pocoo.org/docs/0.12/quickstart/#a-minimal-application)

In [26]:
from flask import Flask, jsonify
app = Flask(__name__)

@app.route('/')
def hello_world():
    #return 'Hello, World!'
    items = [ "c++", "java", "php", "python", "javascript", "asp", "ruby", "perl", "ocaml", "haskell", "rust", "go" ]
    response = jsonify(items)
    response.headers.add('Access-Control-Allow-Origin', '*') #Pour éviter les erreurs de type CORS en dév local
    return response

C'est super mais on n'a plus de sélection parmi les items possibles. Il nous faut récupérer la saisie de l'utilisateur et  faire la sélection côté serveur. 

In [ ]:
from flask import Flask, jsonify, request
app = Flask(__name__)

@app.route('/', methods=['GET'])
def hello_world():
    #return 'Hello, World!'
    response = ""
    term = request.args['term']
    if term:
        items = [ "c++", "java", "php", "python", "javascript", "asp", "ruby", "perl", "ocaml", "haskell", "rust", "go" ]
        response = jsonify([item for item in items if item.startswith(term)]))
        response.headers.add('Access-Control-Allow-Origin', '*') #Pour éviter les erreurs de type CORS en dév local
    return response

## Les données

Maintenant il s'agit de trouver et de préparer un lexique des mots du français (ou de la langue de votre choix).  
Nous devrons répondre à deux questions :
1. dans quel ordre présenter les complétions possibles (alpha, fréquence, autre ?)
2. comment rendre la complétion fonctionnelle avec un volume de données important ?